In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat

print(torch.__version__)
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime

1.8.1
0.8.0a0+e4e171a


In [2]:
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import glob
import random
from torch import nn
import torch.nn.functional as F
from catalyst import dl, utils
from catalyst.data.transforms import ToTensor

In [18]:
from lark.learner import *
from lark.Cnn14_DecisionLevelAtt import *

In [32]:
cfg = Config(site='SSW', use_neptune=True)

In [33]:
cfg.as_dict()

{'site': 'SSW',
 'data_dir': 'data/birdclef-2021',
 'noise_dir': 'data/noise/BirdVox-DCASE-20k/wav-32k',
 'bs': 32,
 'n_workers': 12,
 'training_dataset_size': 3840,
 'duration': 5,
 'use_noise': True,
 'snr_dbs': [10, 3, 1],
 'use_neptune': True,
 'sr': 32000,
 'n_frames': 160000,
 'n_fft': 512,
 'window_length': 512,
 'n_mels': 64,
 'hop_length': 312,
 'f_min': 150,
 'f_max': 15000,
 'labels': ['amecro',
  'amegfi',
  'amerob',
  'balori',
  'belkin1',
  'bkcchi',
  'blujay',
  'cangoo',
  'chswar',
  'comgra',
  'comyel',
  'dowwoo',
  'eastow',
  'eawpew',
  'gockin',
  'grycat',
  'haiwoo',
  'norcar',
  'norfli',
  'norwat',
  'ovenbi1',
  'rebwoo',
  'reevir1',
  'rewbla',
  'sonspa',
  'swaspa',
  'woothr',
  'yebsap']}

In [34]:
vs = ValidDataset(cfg)

In [35]:
vdl = vs.loader

In [36]:
ts = TrainDataset(cfg)

In [37]:
tdl = ts.loader

In [38]:
model = Cnn14_DecisionLevelAtt.load(cfg)

In [39]:
exp = Experiment(cfg)
exp.init(f"pann {datetime.now()}")


psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-7


In [40]:
lr = 1e-4
n_epochs = 10
n_samples = len(tdl)

loss_fn = PANNsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr*10, steps_per_epoch=n_samples, epochs=n_epochs)

In [41]:
model = model.cuda()
loss_fn = loss_fn.cuda()

In [42]:
# from tqdm import tqdm
import sys
from tqdm.notebook import tqdm

def tv_loop(dl, mode):
    if mode == 'train':
        model.train()
    else:
        model.eval()
        
    def inner_loop():
        nonlocal tot_loss
        nonlocal tot_score
        for x, y in pbar:
            x = x.cuda()
            y = y.cuda()
            pred = model(x)
            loss = loss_fn(pred, y)
            score = f1(y, pred, 0.5)
            
            pbar.set_description(f"{mode} loss: {loss:>8f} f1: {score:>8f}")
            exp.log_metric(mode, 'batch', 'loss', loss)
            exp.log_metric(mode, 'batch', 'f1', score)

            tot_loss += loss.item()
            tot_score += score
            if mode == 'train':
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                scheduler.step()
        
    tot_loss = 0
    tot_score = 0
       
    with tqdm(dl, leave=False) as pbar:
        if mode == 'valid':
            with torch.no_grad():
                inner_loop()
        else:
            inner_loop()
            
    n_batches = len(dl)
    tot_loss /= n_batches
    tot_score /= n_batches
    return tot_loss, tot_score

In [43]:
with tqdm(range(n_epochs)) as pbar:
    for epoch in pbar:
        train_loss, train_score = tv_loop(tdl, 'train')
        valid_loss, valid_score = tv_loop(vdl, 'valid')
        msg = f"epoch: {epoch + 1:3d} train loss: {train_loss:>8f} train f1: {train_score:>8f} valid loss: {valid_loss:>8f} valid f1: {valid_score:>8f}"
        print(msg)
        exp.log_metric('train', 'epoch', 'loss', train_loss)
        exp.log_metric('valid', 'epoch', 'loss', valid_loss)
        exp.log_metric('train', 'epoch', 'f1', train_score)
        exp.log_metric('valid', 'epoch', 'f1', valid_score)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:   1 train loss: 0.563347 train f1: 0.038787 valid loss: 0.120235 valid f1: 0.210526


  0%|          | 0/120 [00:00<?, ?it/s]

...

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:   9 train loss: 0.068609 train f1: 0.647612 valid loss: 0.115484 valid f1: 0.268266


  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:  10 train loss: 0.069341 train f1: 0.641824 valid loss: 0.116332 valid f1: 0.270062


In [54]:
with tqdm(vdl, leave=False) as pbar:
    with torch.no_grad():
        for x, y in pbar:
            x = x.cuda()
            y = y.cuda()
            pred = model(x)
            score = f1(y, pred, 0.5)
            print(y)
            print(pred)
            print(score)

  0%|          | 0/38 [00:00<?, ?it/s]

tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
  

KeyboardInterrupt: 